# Using Swig

## fonctions with numpy list parameter

### C header file array_sum.h

In [ ]:
#ifndef __REDUCTION_H__
#define  __REDUCTION_H__

float array_sum(float* tab, int n);

#endif


### C file array_sum.c

In [ ]:
/// Sum of array elements.
/**	@param tab  : float pointer ,    array to sum
 * 	@param n    : unsigned int  ,    array n
 * 	@return     : float         ,    sum of array elements
*/
float array_sum(float* tab, int n){

    float result = 0.;
        for ( unsigned int i =0; i < n; ++i){
            result += tab[i];
        }
        return result;
}


* what exactly *tab* is? Is it a single value to be altered in-place? Is it an array, and if so what is its length?

* Is it input-only? Output-only? Input-output?

* SWIG cannot determine these details

* array_sum is a routine that takes an input-only array of length *n* of float values called *tab* and returns the sum of the array

* For python, the preferred way of handling contiguous  blocks of homogeneous data is with the module NumPy, which provides full object-oriented access to multidimensial arrays of data. 
def array_sum(tab):
"""
array_sum: return the sum of a sequence
array_sum(numpy.ndarray) -> float
array_sum(list) -> float
array_sum(tuple) -> float
"""

* where *tab* would be a NumPy array of float32 values, and its length n would be extracted from *tab* internally before being passed to the C routine
* SWIG allows these types of conversions to be defined via a mechanism called _typemaps

### swig interface file modarray.i

In [2]:
%module modarray

%{
    #define SWIG_FILE_WITH_INIT
    #include "array_sum.h"
%}

%include "numpy.i"

%init %{
    import_array();
%}

%apply (float* IN_ARRAY1, int DIM1) {(float* tab, int n)}

%include "array_sum.h"


IndentationError: unexpected indent (<ipython-input-2-28a17a7d21cf>, line 11)

In [14]:
! swig -python modarray.i

In [15]:
! gcc -fPIC -c array_sum.c modarray_wrap.c -I/home/jacquem/anaconda3/include/python3.5m
! gcc -shared array_sum.o modarray_wrap.o -o _modarray.so

In [20]:
import modarray
import numpy as np

In [22]:
l = np.arange(100, dtype=np.float32)
modarray.array_sum(l)

4950.0

In [23]:
l = [1,2,3,4,5,6,7,8]
modarray.array_sum(l)

36.0

In [24]:
l=(1,2,3,4,5)
modarray.array_sum(l)

15.0

### References

[Swig typemaps ](http://swig.org/Doc1.1/HTML/Typemaps.html).

[Swig interface file for NumPy](https://docs.scipy.org/doc/numpy/reference/swig.interface-file.html).

[Swig Numpy example ](https://scipy.github.io/old-wiki/pages/Cookbook/SWIG_NumPy_examples.html).